In [1]:
import numpy as np
import sys
import hrr_common.utils as cu
from hrr_cobot_robot.gripper_interfaces.wsg_50_hw_local import gripper_command_handle as gch
import hrr_common as hrr_cm
import hrr_cobot_robot as hrr_rob
import spatialmath as sm
import rospy
from tqdm.notebook import tqdm, trange
from hrr_msgs.msg import PlannerAction , PlannerGoal

# ROS
import actionlib

In [2]:
def get_hostname_and_IP():
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        s.connect(("8.8.8.8", 80))
        return socket.gethostname(), s.getsockname()[0]

_host, _ip = get_hostname_and_IP()
%env ROS_MASTER_URI=http://hrrcobotLinux54.lsr.ei.tum.de:11311
# %env ROS_MASTER_URI=http://localhost:11311
print(f"\t\033[1m\033[4m!!!please check that the IPs below is identical!!!\033[0m"+
      f"\n\thost {_host} has IP {_ip}, which should be identical to",end="\nvs:\t")
%env ROS_IP=$_ip

for a local setup, replace the above by:
	%env ROS_MASTER_URI=http://localhost:11311
env: ROS_MASTER_URI=http://hrrN3511rt2004.lsr.ei.tum.de:11311
in case this notebook does not work please check that IP below is identical:
	%env ROS_IP=129.187.147.174		<=>	env: ROS_IP=129.187.147.174


In [3]:
np.set_printoptions(suppress=True)

## Launch

catkin build hrr_ee_tools package

Launch the cobot
```bash
roslaunch hrr_cobot_robot hrr_cobot_hw.launch tool_name:="vacuum"  
```

```bash
rosrun planner planner_server_v2.py 
```
```bash
rosrun hrr_cobot_robot enable_sns_trk --sns_controller_name sensor_track_velocity_controller
```

In [4]:
rospy.init_node('vacuum_grasping_dev')
hrr_rob.load_default_parameters("/hrr_cobot")
#rospy.set_param('/cobot_tool_name', "wsg_50")
cobot = hrr_rob.HrrCobotControl.from_ros()
rospy.set_param("~joint_state_topic_name", "/hrr_cobot/joint_states")
rospy.set_param("~tp5_topic_name", "/hrr_cobot/comau_robot_state_controller/comau_cartesian_data")
comau_status = hrr_rob.ComauRobotState.from_ros()
rospy.sleep(1e-1)
print(comau_status)

[ERROR] [1641980871.027711]: could not set tool-frame @ /vacuum_grasping_dev due to missing ROS-parameter: '~tool_name'


[INFO] [1641980871.511826]: Updated force  gains: K_p:=[0.0001, 0.0001, 0.0001]
[INFO] [1641980871.515564]: Updated torque gains: K_p:=[0.0005, 0.0005, 0.0005]
q:		[   4.09713636    4.22352218 -105.27729153  -88.65102648   93.92755252
  -25.46458276][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[0.45649518 0.04729539 0.65235046][m]
quat:		-0.27107 <[0.27024388 0.65315031 0.6533643 ]>
tool-pos:	[0.45649518 0.04729539 0.65235046][m]
tool-rpy:	[-134.98979187   -0.05507659  -89.95841217][°]
robot-status:	terminate


[WARN] [1641980871.749308]: Could not process inbound connection: Client [/hrr_cobot] wants topic [/hrr_cobot/sensor_track_compliant_controller/hybrid_ctrl_cmd] to have datatype/md5sum [hrr_msgs/HybridForceVelocityCmdStamped/6bfe9d92f9569269ce7086e736ecd0c2], but our version has [hrr_msgs/HybridForceVelocityCmdStamped/c7d32c882fe005053aba472f934842bd] Dropping connection.{'callerid': '/hrr_cobot', 'md5sum': '6bfe9d92f9569269ce7086e736ecd0c2', 'tcp_nodelay': '0', 'topic': '/hrr_cobot/sensor_track_compliant_controller/hybrid_ctrl_cmd', 'type': 'hrr_msgs/HybridForceVelocityCmdStamped'}


### connect vacuum gripper, make sure air is turned on behind the robot

In [5]:
cobot.change_tool("vacuum")


True

In [68]:
cobot.T_B_E_robot

  -0.7615   -0.648     0.01302   0.5001    
  -0.6481    0.7611   -0.02633  -0.3753    
   0.007154 -0.02849  -0.9996    0.4686    
   0         0         0         1         


In [67]:
cobot.T_B_C_robot

  -0.9967    0.08024   0.01302   0.5099    
   0.07988   0.9965   -0.02633  -0.3285    
  -0.01509  -0.02521  -0.9996    0.2693    
   0         0         0         1         


### Force tests

In [59]:
rospy.set_param("~ft_sensor_topic_name", "/hrr_cobot/ft_sensor")
ft_status = hrr_rob.FTData.from_ros()
rospy.sleep(1e-1)
print(ft_status)
ft_status.wrench

current force:	[-9.28619385  5.51879883 17.69238281] [N]
current torque:	[-4.96246338 -3.96057129  0.6362915 ] [Nm]


array([-9.34387207,  5.58325195, 17.58691406, -5.05371094, -3.97338867,  0.65325928])

In [ ]:
%matplotlib widget
cobot.run_calibration_routine(file_name='/tmp/current_tool.npy', plot=True, figsize=(12,10))

In [ ]:
print(ft_status.wrench)

In [ ]:
print(cobot)

In [ ]:
cobot.init_sns_vel()

In [ ]:
cobot.reset()
cobot.set_compliant(np.r_[1.0, 1.0, 1.0, np.zeros(3)], Path("/tmp/current_tool.npy"), F_max=5.0)

In [ ]:
cobot.acknowledge_calibration()

In [ ]:
T = int(10 * cobot.hz)
for t in trange(T):
    cobot.update()

In [ ]:
cobot.reset()
cobot.set_py_hybrid_force_vel_command(B_F_des=np.r_[0.0, 0.0, -10.0, np.zeros(3)], 
                                          K_f=8e-4, K_t=5e-3, scale_pos=0.0, scale_rot=0.0, 
                                      wrench_dir=np.r_[1.0, 1.0, 1.0, np.ones(3)],
                                      vel_dir=np.zeros(6))

### Vacuum tests


In [38]:
from hrr_cobot_robot import ArduinoToolControlInterface
rospy.set_param("~arduino_node_name", "/hrr_cobot/tool_controller")
tool_controller = ArduinoToolControlInterface.from_ros()
tool_controller.tool = "vacuum"
#cobot.set_tool_from_ros_frame(cobot.tool, "vacuum_tip1")

In [59]:
  #Turns vacuum on for 10seconds
tool_controller.vacuum = True

In [57]:
tool_controller.vacuum = False #Turns vacuum off (immediately)

In [ ]:
#Usage fo
for t in trange(20):
    tool_controller.vacuum = True
    rospy.sleep(0.2)

### Use cases
Emergency lamp: 
    Remove and dispose lid after it is cut loose from the fixtures
    Remove and dispose middle cover
Microwave Oven: None
Pc Tower: 
    Dispose PSU after it is pushed from the housing
    Dipose CPU cooler after it is unscrewed


## Params from outside

In [ ]:
from geometry_msgs.msg import PoseStamped #For testing purposes
object_center = PoseStamped() #PoseStamped
release_pose = PoseStamped() #PoseStamped

### What we really need: 
* pose of the robot when activating vacuum gripper (call it goal_pose)
* pose of the robot when deactivating (call it dispose_pose)
* (maybe) a pre-pose or normal vector to the surface

In [42]:
cobot.v_max


0.1

### Load it manually for now

In [58]:
cobot.update_tf()
#np.save('goal_pose_vac',cobot.T_B_E_robot.A,allow_pickle=True,) #saves current pose as goal_pose (for metal sheet of PC)
np.save('goal_pose_vac_PSU',cobot.T_B_E_robot.A,allow_pickle=True,) #saves current pose as goal_pose
goal_pose = np.load('goal_pose_vac_PSU.npy')
sm.SE3(goal_pose)

  -0.7244   -0.6893    0.01283   0.6115    
  -0.6894    0.7243   -0.008773 -0.3118    
  -0.003248 -0.0152   -0.9999    0.3084    
   0         0         0         1         


In [8]:
#np.save('dispose_pose_vac',dispose_pose.A,allow_pickle=True) #saves current pose as release_pose
dispose_pose = np.load('dispose_pose_vac.npy')
sm.SE3(dispose_pose) #prob go down 10-20cm

  -0.6713   -0.7408    0.02192   0.5709    
  -0.7408    0.6716    0.01315   0.3754    
  -0.02447  -0.007406 -0.9997    0.435     
   0         0         0         1         


In [9]:
#np.save('final_pose_vac',final_pose.A,allow_pickle=True)
final_pose = np.load('final_pose_vac.npy')
sm.SE3(final_pose)

  -0.6713   -0.7408    0.02192   0.5709    
  -0.7408    0.6716    0.01315   0.3754    
  -0.02447  -0.007406 -0.9997    0.285     
   0         0         0         1         


In [10]:
#These are used to compute the pre-pose:
nvec = np.array([0, 0, 1])
nvec = cu.normalize_vec(nvec) #Make sure nvec is normalized
pre_pose_dist = 0.04 #in meter

In [11]:
pre_pose = sm.SE3(nvec* pre_pose_dist) @ sm.SE3(goal_pose)
sm.SE3(pre_pose)

  -0.7621   -0.6473    0.01269   0.5003    
  -0.6474    0.7621   -0.00887  -0.3748    
  -0.003926 -0.01497  -0.9999    0.4337    
   0         0         0         1         


In [12]:
cobot.update_tf()
cobot.T_B_E_robot

  -0.707     0.7072   -0.0009613  0.4565    
  -0.001193  0.0001667  1         0.0473    
   0.7072    0.707     0.0007258  0.6524    
   0         0         0         1         


In [17]:
print(cobot)

Hrr-Cobot state:
FT-data:
current force:	[5.97381592 9.78076172 1.13378906] [N]
current torque:	[-4.1973877   0.80108643 -4.64068604] [Nm]
current wrench:	[ 5.95390125  9.79442919  1.17143368 -4.21133579  0.79883407 -4.65334241] [N,Nm]
=>in contact:	True
---
Robot-data:
q:		[   4.09713422    4.22352858 -105.2772847   -88.65102648   93.92755252  -25.46460496][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[0.45649521 0.04729542 0.65235046][m]
quat:		-0.27107 <[0.27024399 0.65315021 0.65336425]>
tool-pos:	[0.45649521 0.04729542 0.65235046][m]
tool-rpy:	[-134.98976135   -0.05508421  -89.95840454][°]
robot-status:	moving
---


### Initiate planner

In [19]:
 # Initialize the planner action client.
    
# >> Remeber to run the planner node (rosrun planner planner_server_v2.py) in terminal
# >> 
#rospy.init_node('planner_client')

client1 = actionlib.SimpleActionClient('planner', PlannerAction)
client1.wait_for_server()

# Sending deride end configuration
goal = PlannerGoal()
goal.goal_pose.header.frame_id = "map"
goal.goal_pose.header.stamp = rospy.Time.now()

## If we want to, we can start at a pose

In [36]:
cobot.move_to_pose( cobot.FK(cobot.q_calib), err_gain = None, v_max = 0.03) #start_pose, cobot.FK(cobot.q_calib)
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1641981515.250526]: reached goal pose at step 621 / 10000


## Go-to pre-pose with traj planner
Assuming pre-pose is a translation of pre_pose_dist in the direction of nvec from goal_pose

In [24]:
pre_pose_quat = cu.homog2quat(pre_pose)
pre_pose_pos = cu.homog2pos(pre_pose)
#position of end_configuration
goal.goal_pose.pose.position.x = pre_pose_pos[0]
goal.goal_pose.pose.position.y = pre_pose_pos[1]
goal.goal_pose.pose.position.z = pre_pose_pos[2]

#orientation of end_configuration
goal.goal_pose.pose.orientation.x = pre_pose_quat.x
goal.goal_pose.pose.orientation.y = pre_pose_quat.y
goal.goal_pose.pose.orientation.z = pre_pose_quat.z
goal.goal_pose.pose.orientation.w = pre_pose_quat.w

In [27]:
client1.send_goal(goal)


cobot.move_to_pose(pre_pose, err_gain = None, v_max = 0.03) #start_pose, cobot.FK(cobot.q_calib)
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

## Approach and grasp

In [28]:
tool_controller.vacuum = True
cobot.move_to_pose(goal_pose, err_gain = None, v_max = 0.03)
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1641981179.201895]: reached goal pose at step 173 / 10000


## Dispose

In [29]:
## Go to pre_pose then dispose location, possible check if object is grasped
cobot.move_to_pose(pre_pose,err_gain = None, v_max = 0.03)
#tool_controller.vacuum = True
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1641981180.977949]: reached goal pose at step 173 / 10000


In [30]:
dispose_pose_quat = cu.homog2quat(dispose_pose)
dispose_pose_pos = cu.homog2pos(dispose_pose)

#position of end_configuration
goal.goal_pose.pose.position.x = dispose_pose_pos[0]
goal.goal_pose.pose.position.y = dispose_pose_pos[1]
goal.goal_pose.pose.position.z = dispose_pose_pos[2]

#orientation of end_configuration
goal.goal_pose.pose.orientation.x = dispose_pose_quat.x
goal.goal_pose.pose.orientation.y = dispose_pose_quat.y
goal.goal_pose.pose.orientation.z = dispose_pose_quat.z
goal.goal_pose.pose.orientation.w = dispose_pose_quat.w

In [31]:
client1.send_goal(goal)

In [32]:
tool_controller.vacuum = True

In [33]:
tool_controller.vacuum = True

#tool_controller.vacuum = True
cobot.move_to_pose(dispose_pose) 
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

### Move down a bit (only for demo) and disable vacuum gripper

In [34]:
tool_controller.vacuum = True
cobot.move_to_pose(final_pose) 
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()
tool_controller.vacuum = False

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1641981192.477557]: reached goal pose at step 319 / 10000


### Go back somewhere